In [9]:
import tensorflow as tf                                   # Used to build and train the CNN model
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt                           # Used to visualize images from the CIFAR-10 dataset
import numpy as np                                        # Handles array operations
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
# Add at the beginning of your notebook
mixed_precision = tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Define image size and batch

batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(

    "C:/Users/umidu/OneDrive/Desktop/AI Project/Dataset/Dataset/train",
    batch_size=batch_size,
    image_size=(128, 128),  

)


val_ds = tf.keras.utils.image_dataset_from_directory(

    "C:/Users/umidu/OneDrive/Desktop/AI Project/Dataset/Dataset/validation",
    batch_size=batch_size,
    image_size=(128, 128),
)


test_ds = tf.keras.utils.image_dataset_from_directory(

    "C:/Users/umidu/OneDrive/Desktop/AI Project/Dataset/Dataset/test",

    batch_size=batch_size,
    image_size=(128, 128),


)

Found 10002 files belonging to 2 classes.
Found 2002 files belonging to 2 classes.
Found 1002 files belonging to 2 classes.


In [11]:
print(train_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
# prompt: make a cnn model for this dataset

# Normalize pixel values to be between 0 and 1
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
# Output layer for binary classification
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use binary cross-entropy for binary classification
              metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
history = model.fit(train_ds, epochs=5,
                    validation_data=val_ds, callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds, verbose=2)
print(f"Test accuracy: {test_acc}")

Epoch 1/5
169/313 ━━━━━━━━━━━━━━━━━━━━ 7:45 3s/step - accuracy: 0.6443 - loss: 0.6584

In [ ]:
# # Performance optimization for the datasets
# AUTOTUNE = tf.data.AUTOTUNE

# # Store the class names before applying transformations
# class_names = ["Real", "Fake"] 

# # Now apply transformations
# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# # Get the input shape
# for images, labels in train_ds.take(1):
#     input_shape = images[0].shape

# # Use the stored class names
# num_classes = len(class_names)
# print(f"Input shape: {input_shape}")
# print(f"Number of classes: {num_classes}")
# print(f"Classes: {class_names}")

In [ ]:
# data_augmentation = tf.keras.Sequential([
#     layers.RandomFlip("horizontal"),
#     layers.RandomRotation(0.1),
#     layers.RandomZoom(0.1),
# ])

In [ ]:


# # Build model with batch normalization and a deeper architecture
# model = models.Sequential([
#     # Data augmentation (only applied during training)
#     data_augmentation,

#     # Normalization layer (ensures consistent input values)
#     layers.Rescaling(1./255, input_shape=(128, 128, 3)),

#     # First convolutional block
#     layers.Conv2D(64, (3, 3), padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Conv2D(64, (3, 3), padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Dropout(0.2),

#     # Second convolutional block
#     layers.Conv2D(128, (3, 3), padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Conv2D(128, (3, 3), padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Dropout(0.3),

#     # Third convolutional block
#     layers.Conv2D(256, (3, 3), padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Conv2D(256, (3, 3), padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Dropout(0.4),

#     # Flatten and dense layers
#     layers.Flatten(),
#     layers.Dense(512),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Dropout(0.5),
#     layers.Dense(2, activation='softmax')
# ])

In [ ]:
# # Compile the model
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

In [ ]:
# # Display the model architecture
# model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_9 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_8 (Rescaling)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# # Set up early stopping to prevent overfitting
# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=3,
#     restore_best_weights=True
# )

In [ ]:
# # Train the model
# history = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=5,
#     callbacks=[early_stopping]
# )

Epoch 1/5


KeyboardInterrupt: 

In [ ]:
# # Evaluate the model on the test dataset
# test_loss, test_acc = model.evaluate(test_ds)
# print(f"Test accuracy: {test_acc:.4f}")

86/86 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - accuracy: 0.4981 - loss: 0.6932
Test accuracy: 0.4964
